# Exploratory Notebook

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../Data/im.db')



In [2]:
df_mb = pd.read_sql("""

SELECT *
FROM movie_basics







""", conn)

In [3]:
df_mb.columns

Index(['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres'],
      dtype='object')

In [4]:
df_mb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [5]:
df_mb['genres'].isna().sum()

5408

In [6]:
columns_to_drop_na = ['movie_id', 'primary_title', 'original_title', 'start_year',
       'runtime_minutes', 'genres']
df_mb.dropna(subset=columns_to_drop_na, inplace=True)


In [7]:
df_mb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112232 entries, 0 to 146139
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         112232 non-null  object 
 1   primary_title    112232 non-null  object 
 2   original_title   112232 non-null  object 
 3   start_year       112232 non-null  int64  
 4   runtime_minutes  112232 non-null  float64
 5   genres           112232 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.0+ MB


In [8]:
df_mb['genres'].isna().sum()

0

In [9]:
df_mr = pd.read_sql("""

SELECT *
FROM movie_ratings







""", conn)

In [10]:
df_mr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [11]:
df_mr.columns

Index(['movie_id', 'averagerating', 'numvotes'], dtype='object')

In [12]:
columns_to_drop_na = ['movie_id', 'averagerating', 'numvotes']
df_mr.dropna(subset=columns_to_drop_na, inplace=True)
df_mr.isna().sum()

movie_id         0
averagerating    0
numvotes         0
dtype: int64

In [13]:
df_mr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ MB


In [14]:
df_ma = pd.read_sql("""

SELECT *
FROM movie_akas


""", conn)

In [15]:
df_ma.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              168447 non-null  object 
 6   attributes         14925 non-null   object 
 7   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


In [16]:
df_ma.columns

Index(['movie_id', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'is_original_title'],
      dtype='object')

In [17]:
#columns_to_drop_na = ['movie_id', 'ordering', 'title', 'region', 'language', 'types',
#       'attributes', 'is_original_title']
#df_ma.dropna(subset=columns_to_drop_na, inplace=True)

In [18]:
df_ma.isna().sum()


movie_id                  0
ordering                  0
title                     0
region                53293
language             289988
types                163256
attributes           316778
is_original_title        25
dtype: int64

In [19]:
df_5a = pd.read_sql("""

SELECT movie_id, primary_title, genres, MAX (averagerating) AS av_rating , MAX(numvotes) AS num_votes
FROM movie_basics
    JOIN movie_ratings 
       USING(movie_id)
        GROUP BY movie_id
        ORDER BY num_votes DESC;



""", conn)

In [20]:
df_5a.head(10)

,movie_id,primary_title,genres,av_rating,num_votes
0,tt1375666,Inception,"Action,Adventure,Sci-Fi",8.8,1841066
1,tt1345836,The Dark Knight Rises,"Action,Thriller",8.4,1387769
2,tt0816692,Interstellar,"Adventure,Drama,Sci-Fi",8.6,1299334
3,tt1853728,Django Unchained,"Drama,Western",8.4,1211405
4,tt0848228,The Avengers,"Action,Adventure,Sci-Fi",8.1,1183655
5,tt0993846,The Wolf of Wall Street,"Biography,Crime,Drama",8.2,1035358
6,tt1130884,Shutter Island,"Mystery,Thriller",8.1,1005960
7,tt2015381,Guardians of the Galaxy,"Action,Adventure,Comedy",8.1,948394
8,tt1431045,Deadpool,"Action,Adventure,Comedy",8.0,820847
9,tt1392170,The Hunger Games,"Action,Adventure,Sci-Fi",7.2,795227


In [21]:
df_5a['genres'] = df_5a['genres'].str.split(',') 


In [22]:
df_5a = df_5a[df_5a['genres'].notnull()]

In [23]:
df_5a = df_5a.explode('genres')

In [24]:
#df_5a = df_5a.drop_duplicates(subset=['primary_title'])

In [26]:
df_5a.to_sql('cleaned_movies', conn, if_exists='replace')

In [27]:
df_5a.head(20)

,movie_id,primary_title,genres,av_rating,num_votes
0,tt1375666,Inception,Action,8.8,1841066
0,tt1375666,Inception,Adventure,8.8,1841066
0,tt1375666,Inception,Sci-Fi,8.8,1841066
1,tt1345836,The Dark Knight Rises,Action,8.4,1387769
1,tt1345836,The Dark Knight Rises,Thriller,8.4,1387769
2,tt0816692,Interstellar,Adventure,8.6,1299334
2,tt0816692,Interstellar,Drama,8.6,1299334
2,tt0816692,Interstellar,Sci-Fi,8.6,1299334
3,tt1853728,Django Unchained,Drama,8.4,1211405
3,tt1853728,Django Unchained,Western,8.4,1211405


In [28]:
df_5a.to_csv('movie_numvotes.csv')

In [ ]:


df_5b = pd.read_sql("""

SELECT movie_id, primary_title, genres, MAX (averagerating) AS av_rating , MAX(numvotes) AS num_votes
FROM movie_basics
    JOIN movie_ratings 
       USING(movie_id)
        GROUP BY movie_id
        ORDER BY av_rating DESC;



""", conn)

In [ ]:
df_5b.head(20)

In [ ]:
import matplotlib.pyplot as plt


# Plot the 'num_votes' column
plt.plot(df_5['num_votes'], marker='o', linestyle='-')

# Add labels and title
plt.xlabel('Index')
plt.ylabel('Number of Votes')
plt.title('popularity by num_votes')

# Show the plot
plt.show()


In [ ]:
df_5['genres'].values

In [ ]:
df_5['genres'].values

In [ ]:

# bar chart 
fig, ax = plt.subplots(figsize=(12, 8))
ax.bar(df_5['genres'].values, df_5['num_votes'])
ax.set_ylabel('Number of Movies')
ax.set_title('Popularity of movies by Genre')
ax.set_xticklabels(df_5['genres'].index, rotation=45, ha='right')

# show plot
plt.show()

In [ ]:
df_mg = pd.read_csv('../Data/bom.movie_gross.csv.gz')

In [ ]:
df_mg

In [ ]:
df_mg.info()

In [ ]:
df_mg.columns

In [ ]:
df_mg.isna().sum()

In [ ]:
columns_to_drop_na = ['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']
df_mg.dropna(subset=columns_to_drop_na, inplace=True)
df_mg.isna().sum()

In [ ]:
df_mg.info()

In [ ]:
df_mg

In [ ]:
df_mg['domestic_gross'] = pd.to_numeric(df_mg['domestic_gross'], errors='coerce').fillna(0).astype(int)
df_mg['foreign_gross'] = pd.to_numeric(df_mg['foreign_gross'], errors='coerce').fillna(0).astype(int)

In [ ]:
df_mg.info()

In [ ]:
df_mg['total_gross'] = df_mg['domestic_gross'] + df_mg['foreign_gross']

In [ ]:
df_gross= df_mg.head(20)

In [ ]:
df_gross

In [ ]:
df_gross.to_csv('movie_gross.csv')

In [ ]:
df_rt = pd.read_csv('../Data/rt.reviews.tsv.gz', sep='\t', encoding='latin-1')

In [ ]:
df_rt.info()